In [51]:
## radom forest par remplissage avec ce qu'il y avait avant !!!!

import os
import sys
parent_dir = os.path.dirname(os.getcwd())
sys.path.append(parent_dir)
utils_dir = os.path.join(parent_dir, "src", "utils")
sys.path.append(utils_dir)

In [52]:
import pandas as pd
import plotly.graph_objects as go
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
#from tensorflow.keras.models import Sequential
#from tensorflow.keras.layers import GRU, Dense, Dropout
#from tensorflow.keras.optimizers import Adam
#from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
#from process_data import process_data
#from weighted_accuracy import weighted_accuracy

In [53]:
X = os.path.join("..", "data","original_input", "X_train_Wwou3IE.csv")
Y = os.path.join("..", "data","original_input", "X_test_GgyECq8.csv")
#Z = os.path.join("..", "data","original_input", "y_random_pt8afo8.csv")
W = os.path.join("..", "data","original_input", "y_train_jJtXgMX.csv")

X_train = pd.read_csv(X, delimiter=',')
X_test = pd.read_csv(Y,delimiter=',')
delivery_start = X_test["DELIVERY_START"]
y_train = pd.read_csv(W, delimiter=',')
#y_random = pd.read_csv(Z, delimiter=',')

In [54]:
X_train['DELIVERY_START'] = pd.to_datetime(X_train['DELIVERY_START'], utc = True)
X_train.set_index('DELIVERY_START', inplace=True)
split_index = pd.to_datetime('2022-06-30 23:00:00+00:00')
# Split the DataFrame into two parts
# df1 will contain rows up to and including the split_index
df1 = X_train.loc[:split_index]

# df2 will contain rows after the split_index
df2 = X_train.loc[split_index + pd.Timedelta(seconds=1):]
X_train_before = X_train.copy(deep=True)
df1_clean = df1.dropna(subset=['load_forecast'])
df2_clean = df2.dropna(subset=['load_forecast'])
n_to_replace = len(X_train) - len(df1_clean) - len(df2_clean)
len(df1_clean), n_to_replace
start_index_for_replacement_values = len(df1_clean) - n_to_replace
end_index_for_replacement_values = len(df1_clean)

# Calculate start and end indices for the missing values
start_index_for_missing_values = len(df1_clean)
end_index_for_missing_values = len(df1_clean) + n_to_replace

# Select the replacement values
replacement_values = X_train.iloc[start_index_for_replacement_values:end_index_for_replacement_values]['load_forecast'].values

# Replace the missing values
X_train.iloc[start_index_for_missing_values:end_index_for_missing_values, X_train.columns.get_loc('load_forecast')] = replacement_values
#X_train_preprocessed.to_csv("../data/original_input/X_train_filled.csv", index = True)
X_train.reset_index(inplace=True)

In [55]:
X_train.set_index("DELIVERY_START", inplace=True)
X_test.set_index("DELIVERY_START", inplace=True)
y_train.set_index("DELIVERY_START", inplace=True)
X_train.index = pd.to_datetime(X_train.index, utc=True)
X_test.index = pd.to_datetime(X_test.index, utc=True)
y_train.index = pd.to_datetime(y_train.index, utc=True)

In [56]:
# gérer les NaN et predicted spot price

X_train = X_train.drop(columns=["predicted_spot_price"]);
X_train = X_train.fillna(X_train.mean())

X_test = X_test.drop(columns=["predicted_spot_price"]);
X_test = X_test.fillna(X_test.mean())

In [57]:
from sklearn.preprocessing import StandardScaler

scaler_model = StandardScaler()
original_index = X_train.index
original_columns = X_train.columns
X_train = scaler_model.fit_transform(X_train)
X_train = pd.DataFrame(X_train, index=original_index, columns=original_columns)

In [58]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor

In [59]:
# Attention  minutes de calcul
# Modèle Random Forest
tree_model = RandomForestRegressor(random_state=42)
# Grille d'hyperparamètres à rechercher
param_grid_random_forest = {
    'n_estimators': [50,100,150],  # Nombre d'arbres dans la forêt
    #'criterion': ['squared_error', 'poisson', 'absolute_error', 'friedman_mse'],  # Fonction de mesure de qualité utilisée pour diviser les nœuds
    #'max_depth': [None,5,10,15,20],  # Profondeur maximale de chaque arbre
    'min_samples_split': [2, 5, 10, 15],  # Nombre minimal d'échantillons requis pour diviser un nœud interne
    #'min_samples_leaf': [1, 2, 4, 6],  # Nombre minimal d'échantillons requis pour être une feuille
    #'max_features': ['auto', 'sqrt', 'log2', None]  # Nombre maximal de fonctionnalités à considérer pour la division
}



# Créez une instance GridSearchCV avec validation croisée
grid_search_random_tree = GridSearchCV(tree_model, param_grid_random_forest, cv=5, scoring='accuracy')

# Effectuer la recherche des meilleurs hyperparamètres
grid_search_random_tree.fit(X_train, y_train)

# Afficher les meilleurs hyperparamètres et la meilleure précision
best_params_tree = grid_search_random_tree.best_params_
best_accuracy_tree = grid_search_random_tree.best_score_

print("Meilleurs hyperparamètres pour Decision Tree:", best_params_tree)
print("Meilleure précision pour Decision Tree:", best_accuracy_tree)

c:\Users\emac\Documents\MachineLearningAvance\elmy\elmy\elmy_env\lib\site-packages\sklearn\base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Users\emac\Documents\MachineLearningAvance\elmy\elmy\elmy_env\lib\site-packages\sklearn\model_selection\_validation.py:821: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\emac\Documents\MachineLearningAvance\elmy\elmy\elmy_env\lib\site-packages\sklearn\model_selection\_validation.py", line 810, in _score
    scores = scorer(estimator, X_test, y_test)
  File "c:\Users\emac\Documents\MachineLearningAvance\elmy\elmy\elmy_env\lib\site-packages\sklearn\metrics\_scorer.py", line 266, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_tr

Meilleurs hyperparamètres pour Decision Tree: {'min_samples_split': 2, 'n_estimators': 50}
Meilleure précision pour Decision Tree: nan


In [60]:
#best_params_forest = {'max_depth': None,
# 'min_samples_leaf': 1,
# 'min_samples_split': 2,
# 'n_estimators': 50}

In [61]:
X_test.isna().sum()

load_forecast                    0
coal_power_available             0
gas_power_available              0
nucelear_power_available         0
wind_power_forecasts_average     0
solar_power_forecasts_average    0
wind_power_forecasts_std         0
solar_power_forecasts_std        0
dtype: int64

In [62]:
tree_model = RandomForestRegressor(**best_params_tree, random_state=42)
tree_model.fit(X_train, y_train)
y_train_pred_tree = tree_model.predict(X_train)

c:\Users\emac\Documents\MachineLearningAvance\elmy\elmy\elmy_env\lib\site-packages\sklearn\base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


In [63]:
y_pred = tree_model.predict(X_test)
print(type(y_pred))

<class 'numpy.ndarray'>


In [64]:
d = {"DELIVERY_START": delivery_start, "spot_id_delta": y_pred.reshape(-1)}
y_pred = pd.DataFrame(data=d)

#y_pred.to_csv("../data/output/y_test_Random_forest_all_data_not_to_filled.csv", index = False)